<a href="https://colab.research.google.com/github/yunusemravci/utdf2gmns/blob/main/intersection_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install utdf2gmns
!pip install osm2gmns

In [ ]:
from IPython.utils import path
from traitlets.traitlets import link
from osm2gmns.osmnet.build_net import getNetFromFile
import os
import osm2gmns as og
import pandas as pd
import csv
from collections import defaultdict

In [4]:
def generate_intersection_codes(df):
  node_id_group = df.groupby(['node_id'])
  intersection_codes = {}

  for node_id, group in node_id_group:

    codes = []
    for (ib_node_id,mvmt_txt), sub_group in group.groupby(['ib_osm_node_id', group['mvmt_txt_id'].str[:-1]]):

      lane_type = mvmt_txt[-1]
      direction = mvmt_txt
      lane_types = sub_group['mvmt_txt_id'].str[-1].unique()

      unique_ib_osm_node_ids = sub_group['ib_osm_node_id'].nunique()
      unique_start_ib_lanes = sub_group['start_ib_lane'].nunique()
      unique_end_ib_lanes = sub_group['end_ib_lane'].nunique()
      total_lanes = sub_group['lanes'].sum()

      if len(lane_types) > 1:
        # All lanes are shared in sub group
        if unique_ib_osm_node_ids == 1 and unique_start_ib_lanes == 1 and unique_end_ib_lanes == 1:
          # Check any shared lanes with thru type
          if 'T' in lane_types:
            shared_type = [t for t in lane_types if t != 'T'][0]
            code =  f"{shared_type}S{sub_group[sub_group['mvmt_txt_id'].str[-1] == 'T']['lanes'].sum()}"
          # No thru type, merge other shared lane types
          else:
            shared_types = ''.join(sorted(lane_types))
            code = f"{shared_types}S{sub_group['end_ib_lane'].sum() - sub_group['start_ib_lane'].sum() + 1}"
        # some of lanes are shared of thru type's with other types
        elif unique_ib_osm_node_ids == 1 and unique_start_ib_lanes == 1 and unique_end_ib_lanes != 1:
          thru_lanes = sub_group[sub_group['mvmt_txt_id'].str[-1] == 'T']['lanes'].sum()
          shared_lanes = total_lanes - thru_lanes
          # Find shared type with thru type
          shared_type = [t for t in lane_types if t != 'T'][0]

          code = f"T{thru_lanes-shared_lanes}{shared_type}S{shared_lanes}"
        elif unique_ib_osm_node_ids == 1 and unique_start_ib_lanes != 1 and unique_end_ib_lanes != 1:
          code = ""
          for i in range(len(lane_types)):
            current_lane_type = lane_types[i]

            current_lane_count = sub_group[sub_group['mvmt_txt_id'].str.endswith(current_lane_type)]['lanes'].sum()

            code += f"{current_lane_type}{current_lane_count}"

        codes.append(f"{direction}{code}")
      else:
        codes.append(f"{direction}{lane_types[0]}{total_lanes}")

    intersection_codes[node_id] = sorted(codes)

  return intersection_codes

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/utdf2gmns/data_bullhead_seg4/movement_utdf.csv')

intersection_codes = generate_intersection_codes(df)
#print(intersection_codes)

combined_codes = {}
for key, value in intersection_codes.items():
    combined_string = ''.join(sorted(value))
    combined_codes[key] = combined_string

# Mapping osm_node_id and merged intersection codes
df['intersection_code'] = df['node_id'].map(combined_codes)

# Keep only one intersection record for the repeating node_id
df.loc[df['node_id'].duplicated(keep='first'), 'intersection_code'] = None

# Save back to movement_utdf
df.to_csv("/content/drive/MyDrive/utdf2gmns/data_bullhead_seg4/movement_utdf_with_codes.csv", index=False)
